In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchaudio
import pandas as pd
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import IPython.display as ipd
from datetime import datetime
import time

from torch.utils.data import DataLoader
from torchmetrics.classification import MulticlassAccuracy, MultilabelAccuracy
import audiomentations
from torch.utils.data import default_collate
from torchvision.transforms import v2
import timm

from src.audio_utils import play_audio, plot_specgram, plot_waveform
from src.data import AudioDataset, FrequencyMaskingAug, TimeMaskingAug, CutMix, MixUp
from src.data_utils import get_metadata, get_fold, get_metadata_from_csv
from src.train_utils import FocalLoss, BCEFocal2WayLoss, get_cosine_schedule_with_warmup, wandb_init, train_one_epoch, eval_one_epoch
from src.models import BasicClassifier, GeMClassifier, SEDClassifier
from src.utils import score_np, roc_auc

import ast
import wandb
import yaml

/home/cedric/miniconda3/envs/birds/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.efficientat.dymn.model import get_model as get_dymn
model = get_dymn(width_mult=1., pretrained_name="dymn10_as", num_classes=182, features_only=True, out_indices=(1,2,3,4,5,6,10,12,15,16))

/home/cedric/miniconda3/envs/birds/lib/python3.10/site-packages/torchvision/ops/misc.py:120: UserWarning: Don't use ConvNormActivation directly, please use Conv2dNormActivation and Conv3dNormActivation instead.
  warnings.warn(
Downloading: "https://github.com/fschmid56/EfficientAT/releases/download/v0.0.1/dymn10_as.pt" to resources/dymn10_as.pt
100%|██████████| 40.5M/40.5M [00:06<00:00, 6.47MB/s]


Features only is true, Dropping final fully-connected layer and loading weights in non-strict mode
DyMN(
  (layers): ModuleList(
    (0): DY_Block(
      (exp_conv): DynamicWrapper(
        (module): Identity()
      )
      (exp_norm): Identity()
      (exp_act): DynamicWrapper(
        (module): Identity()
      )
      (depth_conv): DynamicConv(
        (residuals): Sequential(
          (0): Linear(in_features=32, out_features=4, bias=True)
        )
      )
      (depth_norm): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (depth_act): DyReLUB(
        (coef_net): Sequential(
          (0): Linear(in_features=32, out_features=64, bias=True)
        )
        (sigmoid): Sigmoid()
      )
      (ca): CoordAtt()
      (proj_conv): DynamicConv(
        (residuals): Sequential(
          (0): Linear(in_features=32, out_features=4, bias=True)
        )
      )
      (proj_norm): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_

In [3]:
a = torch.randn((2,1,128,128))
preds, features = model(a)

In [5]:
for f in features:
    print(f.shape)

torch.Size([2, 16, 64, 64])
torch.Size([2, 24, 32, 32])
torch.Size([2, 24, 32, 32])
torch.Size([2, 40, 16, 16])
torch.Size([2, 40, 16, 16])
torch.Size([2, 40, 16, 16])
torch.Size([2, 80, 8, 8])
torch.Size([2, 112, 8, 8])
torch.Size([2, 160, 4, 4])
torch.Size([2, 960, 4, 4])


In [4]:
preds.shape

torch.Size([2, 960, 4, 4])

In [11]:
from src.efficientat.mn.model import get_model as get_mn
model = get_mn(width_mult=1., pretrained_name="mn10_as", num_classes=182, features_only=True, out_indices=(1,3,6,10,12,15,16))

Features only is true, Dropping final fully-connected layer and loading weights in non-strict mode
MN(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvN

/home/cedric/miniconda3/envs/birds/lib/python3.10/site-packages/torchvision/ops/misc.py:120: UserWarning: Don't use ConvNormActivation directly, please use Conv2dNormActivation and Conv3dNormActivation instead.
  warnings.warn(


In [12]:
a = torch.randn((2,1,128,128))
preds, features = model(a)

In [13]:
preds.shape

torch.Size([2, 960, 4, 4])

In [14]:
for f in features:
    print(f.shape)

torch.Size([2, 16, 64, 64])
torch.Size([2, 24, 32, 32])
torch.Size([2, 40, 16, 16])
torch.Size([2, 80, 8, 8])
torch.Size([2, 112, 8, 8])
torch.Size([2, 160, 4, 4])
torch.Size([2, 960, 4, 4])


### Config

In [2]:
class Config:
    start_idx = 'first',
    duration = 5
    sample_rate = 32000
    target_length = 256
    n_mels = 256
    n_fft = 2028
    window = 2028
    audio_len = duration*sample_rate
    hop_length = audio_len // (target_length-1)
    fmin = 20
    fmax = 16000
    top_db = 80

    n_classes = 182
    batch_size = 20
    Model = SEDClassifier
    model_name = 'tf_efficientnetv2_s'
    n_folds = 5
    upsample_thr = 25
    use_class_weights = True   # Test

    standardize = False
    dataset_mean = [-16.8828]
    dataset_std = [12.4019]

    data_aug = True     # Test     
    cutmix_mixup = False     # Test
    loss = 'crossentropy'    # Test ('crossentropy', 'bce')
    secondary_labels_weight = 0.3   # Test (0)
    use_focal = False    # Test (only with bce)
    use_2wayfocal = False
    focal_gamma = 2
    focal_lambda = 1
    label_smoothing = 0.05  # Only with crossentropy

    folds = [2,3,4]
    num_epochs = 9
    warmup_epochs = 1
    lr = 1e-4
    start_lr = 0.01 # relative to lr
    final_lr = 0.01
    weight_decay = 0.0001
    max_grad_norm = 10

    wandb = True
    competition   = 'birdclef-2024' 
    _wandb_kernel = 'cvincent13'
    date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    run_name = f"{date}_{n_mels}x{target_length}_{model_name}"
    wandb_group = 'FullCV'

    base_dir = ''
    short_noises = 'data/background/esc50/use_label'
    background_noises = ['data/background/aicrowd2020_noise_30sec/noise_30sec',
                         'data/background/ff1010bird_nocall/nocall',
                         #'data/background/train_soundscapes/nocall'
                        ]

#metadata = get_metadata(Config.n_folds)
metadata = get_metadata_from_csv('metadata.csv', 'data')

### Dataset

In [3]:
# Data transforms and augmentations
waveform_transforms = audiomentations.Compose([
    audiomentations.Shift(min_shift=-0.5, max_shift=0.5, p=0.5),
    audiomentations.SevenBandParametricEQ(min_gain_db=-12., max_gain_db=12., p=0.2),
    audiomentations.AirAbsorption(min_temperature=10, max_temperature=20, min_humidity=30, max_humidity=90,
                                  min_distance=10, max_distance=100, p=0.8), 

    audiomentations.OneOf([
        audiomentations.Gain(min_gain_db=-4., max_gain_db=4., p=1), 
        audiomentations.GainTransition(min_gain_db=-12., max_gain_db=3., p=1)
    ], p=0.5),

    audiomentations.OneOf([
        audiomentations.AddGaussianSNR(min_snr_db=5., max_snr_db=40., p=1.),
        audiomentations.AddColorNoise(min_snr_db=5., max_snr_db=40., min_f_decay=-3.01, max_f_decay=-3.01, p=1.)
    ], p=0.5),

    audiomentations.AddShortNoises(sounds_path=Config.short_noises, min_snr_db=5., max_snr_db=30., 
                               noise_rms='relative_to_whole_input',
                               min_time_between_sounds=2., max_time_between_sounds=8., 
                               noise_transform=audiomentations.PolarityInversion(), p=0.4),
    audiomentations.AddBackgroundNoise(sounds_path=Config.background_noises, min_snr_db=5., max_snr_db=30., 
                                   noise_transform=audiomentations.PolarityInversion(), p=0.4),
                                   
    audiomentations.LowPassFilter(min_cutoff_freq=750., max_cutoff_freq=7500., min_rolloff=12, max_rolloff=24, p=0.5),
    audiomentations.PitchShift(min_semitones=-2.5, max_semitones=2.5, p=0.3)
])

spec_transforms = nn.Sequential(
    FrequencyMaskingAug(0.4, 0.1, Config.n_mels, n_masks=3, mask_mode='mean'),
    TimeMaskingAug(0.4, 0.1, Config.target_length, n_masks=3, mask_mode='mean'),
)

waveform_transforms=None if not Config.data_aug else waveform_transforms
spec_transforms=None if not Config.data_aug else spec_transforms


cutmix_or_mixup = v2.RandomApply([
    v2.RandomChoice([
        CutMix(num_classes=Config.n_classes, alpha=0.5, one_hot_labels=Config.loss=='bce'),
        MixUp(num_classes=Config.n_classes, alpha=0.5, one_hot_labels=Config.loss=='bce')
    ], p=[0.65, 0.35])
], p=0.5)


def mix_collate_fn(batch):
    return cutmix_or_mixup(*default_collate(batch))

collate_fn = mix_collate_fn if Config.cutmix_mixup else None


### Training

In [4]:
device = torch.device('cuda')

for fold in Config.folds:
    print(f"Training fold {fold}")
    # Read data
    train_df, valid_df, class_weights = get_fold(metadata, fold, up_thr=Config.upsample_thr)

    # Dataloaders
    train_dataset = AudioDataset(train_df, Config, waveform_transforms=waveform_transforms, spec_transforms=spec_transforms)
    val_dataset = AudioDataset(valid_df, Config, waveform_transforms=None, spec_transforms=None)
    train_loader = DataLoader(train_dataset, batch_size=Config.batch_size, shuffle=True, num_workers=4, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=Config.batch_size, shuffle=False, num_workers=4)

    # Model and optim
    model = Config.Model(Config.n_classes, Config.model_name, n_mels=Config.n_mels).to(device)
    optimizer = torch.optim.Adam(model.parameters(), weight_decay=Config.weight_decay, lr=Config.lr)
    spe = len(train_loader)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=spe*Config.warmup_epochs, num_training_steps=spe*Config.num_epochs, 
                                                start_lr=Config.start_lr, final_lr=Config.final_lr)
    
    # Losses and metrics
    pos_weight = torch.tensor(class_weights).to(device) if Config.use_class_weights else None
    if Config.loss == 'crossentropy':
        criterion = nn.CrossEntropyLoss(label_smoothing=Config.label_smoothing, weight=pos_weight)
        accuracy = MulticlassAccuracy(num_classes=Config.n_classes).to(device)
    elif Config.loss == 'bce':
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight, weight=None)
        accuracy = MultilabelAccuracy(num_labels=Config.n_classes).to(device)

    focal_criterion = FocalLoss(gamma=Config.focal_gamma, pos_weight=pos_weight)
    focal2way_criterion = BCEFocal2WayLoss(gamma=Config.focal_gamma, pos_weight=pos_weight)

    # Start training
    start_time = time.time()
    if Config.wandb:
        run = wandb_init(fold, Config)

    save_dir = f"{Config.base_dir}checkpoints/{Config.run_name}" + f"_fold-{fold}"
    train_losses = []
    val_losses = []
    train_metrics = {'AUC': [], 'Accuracy': [], 'Score': []}
    val_metrics = {'AUC': [], 'Accuracy': [], 'Score': []}

    for epoch in range(Config.num_epochs):
        train_loss, train_accuracy, gt, preds = train_one_epoch(Config, model, train_loader, device, optimizer, scheduler, 
                                                                criterion, accuracy, focal_criterion, focal2way_criterion)
        train_losses.append(train_loss)
        train_metrics["Accuracy"].append(train_accuracy)
        train_auc = roc_auc(preds, gt)
        train_score = score_np(preds, gt)
        train_metrics["AUC"].append(train_auc)
        train_metrics["Score"].append(train_score)


        val_loss, val_accuracy, gt, preds = eval_one_epoch(Config, model, val_loader, device, criterion, 
                                                         accuracy, focal_criterion, focal2way_criterion)
        val_losses.append(val_loss)
        val_metrics['Accuracy'].append(val_accuracy)
        val_auc = roc_auc(preds, gt)
        val_score = score_np(preds, gt)
        val_metrics['AUC'].append(val_auc)
        val_metrics['Score'].append(val_score)

        save_dict = {
            "model": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
            "epoch": epoch+1,
            "train_losses": train_losses,
            "train_metrics": train_metrics,
            "val_losses": val_losses,
            "val_metrics": val_metrics
        }

        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        torch.save(save_dict, save_dir + "/checkpoint.pth")
        with open(save_dir + "/logs.txt", "w") as f:
            f.write(f"Epoch {epoch+1}: Train Loss = {train_loss:.3f} | Val Loss = {val_loss:.3f}")
            f.write("\n")
            f.write("CONFIG:")
            for k,v in dict(vars(Config)).items():
                if '__' not in k:
                    f.write("\n")
                    f.write(f"{k}: {v}")


        if Config.wandb:
            wandb.log({
                "train_loss": train_loss,
                "train accuracy": train_accuracy,
                "train_auc": train_auc,
                "train_score": train_score,
                "val_loss": val_loss,
                "val_accuracy": val_accuracy,
                "val_auc": val_auc,
                "val_score": val_score,
                "lr": scheduler.get_last_lr()
            })


        print(f'Epoch {epoch+1}: Train Loss = {train_loss:.3f}, Train Accuracy = {train_accuracy:.3f}, Train ROCAUC = {train_auc:.3f},\
    Train score = {train_score:.3f} | Val Loss = {val_loss:.3f}, Val Accuracy = {val_accuracy:.3f}, \
    Val ROCAUC = {val_auc:.3f}, Val score = {val_score:.3f}')


    def format_duration(seconds):
        hours, remainder = divmod(seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        return "{:02}h {:02}min {:02}s".format(int(hours), int(minutes), int(seconds))

    print(f'Done in {format_duration(time.time() - start_time)}')

    if Config.wandb:
        #print('# WandB')
        #log_wandb(valid_df)
        wandb.run.finish()

Training fold 2
Num Train: 20162, 182 classes | Num Valid: 4892, 182 classes


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cvincent13 (667). Use `wandb login --relogin` to force relogin


val loss: 3.273: 100%|██████████| 245/245 [01:19<00:00,  3.06it/s]


Epoch 1: Train Loss = 5.559, Train Accuracy = 0.000, Train ROCAUC = 0.539,    Train score = 0.539 | Val Loss = 4.609, Val Accuracy = 0.000,     Val ROCAUC = 0.763, Val score = 0.763


val loss: 2.061: 100%|██████████| 245/245 [01:17<00:00,  3.14it/s]


...

val loss: 2.110: 100%|██████████| 245/245 [01:19<00:00,  3.07it/s]


Epoch 8: Train Loss = 2.277, Train Accuracy = 0.002, Train ROCAUC = 0.978,    Train score = 0.978 | Val Loss = 2.243, Val Accuracy = 0.000,     Val ROCAUC = 0.959, Val score = 0.959


val loss: 2.096: 100%|██████████| 245/245 [01:19<00:00,  3.07it/s]


Epoch 9: Train Loss = 2.214, Train Accuracy = 0.002, Train ROCAUC = 0.981,    Train score = 0.981 | Val Loss = 2.229, Val Accuracy = 0.000,     Val ROCAUC = 0.961, Val score = 0.961
Done in 01h 43min 50s


train accuracy,▁▁▄▃▅▆▆▇█
train_auc,▁▅▇▇█████
train_loss,█▆▄▃▂▂▁▁▁
train_score,▁▅▇▇█████
val_accuracy,▁▃▃▄▄▅█▆▆
val_auc,▁▆▇██████
val_loss,█▄▃▂▂▁▁▁▁
val_score,▁▆▇██████
train accuracy,0.0019
train_auc,0.98057
train_loss,2.21388


# Optimize for inference

In [8]:
import openvino as ov
import nncf

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, openvino


In [8]:
# Load checkpoint
device = torch.device('cpu')

model = Config.Model(Config.n_classes, pretrained=False, model_name=Config.model_name, n_mels=Config.n_mels).to(device)

#save_dir = f"{Config.base_dir}checkpoints/{Config.run_name}"
save_dir = 'checkpoints/2024-05-25_01-03-08_256x256_tf_efficientnetv2_s_fold-4'
checkpoint_name = f'{save_dir}/checkpoint.pth'
checkpoint_ov = f'{save_dir}/checkpoint.xml'
    
checkpoint = torch.load(checkpoint_name, map_location='cpu')
model.load_state_dict(checkpoint['model'])
model.eval()

SEDClassifier(
  (bn0): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): EfficientNetFeatures(
    (conv_stem): Conv2dSame(3, 24, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNormAct2d(
      24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (drop_path): Identity()
        )
        (1): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            24, eps=0.001, momentum=0.1, affine=True, tr

In [9]:
# Convert and save model for openvino
input_data = torch.rand(1, 3, Config.n_mels, Config.target_length)
ov_model = ov.convert_model(model, example_input=input_data)
ov.save_model(ov_model, save_dir + '/checkpoint.xml')

In [13]:
# Read and compile model with openvino
core = ov.Core()
ov_model = core.read_model(save_dir + "/checkpoint.xml")
compiled_model = ov.compile_model(ov_model)

In [12]:
# Quantize model to 8 bits openvino

val_loader

In [11]:
target = torch.zeros((8,182))
input = torch.zeros((8,182))